<a href="https://colab.research.google.com/github/davidlealo/100profes/blob/master/resumen_libro_grande.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Configurar el entorno en Google Colab


In [1]:
!pip install openai langchain PyPDF2 transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.5 MB/s eta 0:00:00


In [ ]:
from google.colab import files

uploaded = files.upload()

for filename in uploaded.keys():
    print(f"Archivo subido: {filename}")


In [ ]:
from PyPDF2 import PdfReader

def extract_text_from_pdf(pdf_path):
    reader = PdfReader(pdf_path)
    text = ""
    for page in reader.pages:
        text += page.extract_text()
    return text

file_path = list(uploaded.keys())[0]
book_text = extract_text_from_pdf(file_path)
print(f"El libro tiene {len(book_text)} caracteres.")
